In [2]:
import math
import os
from os import path
from pathlib import Path
from django.db.models import FloatField, F, Q

from captions import CaptionIndex, Documents, Lexicon   # type: ignore

from typing import Callable, Dict, List, Set, Tuple, NamedTuple, Union

# from rekall.temporal_predicates import *
# from rekall.spatial_predicates import *
# from rekall.interval_list import IntervalList
from rekall import IntervalSetMapping
from rekall.stdlib.ingest import ism_from_django_qs

# from esper.rekall import *
# from esper.interview import *
# from esper.captions import *
# from esper.prelude import esper_widget

# from query.models import LabeledInterview, LabeledPanel, LabeledCommercial, Video, FaceIdentity, Face
from app.models import Video, Face, FaceIdentity, LabeledInterview, CanonicalShow, Commercial

In [3]:
def ground_truth_interviews(name, original = True):
    interviews = LabeledInterview.objects \
        .annotate(fps = F('video__fps')) \
        .annotate(min_frame = F('fps') * F('start')) \
        .annotate(max_frame = F('fps') * F('end')) \
        .filter(guest1 = name)
    if original:
        interviews = interviews.filter(original = original)
    return IntervalSetMapping(ism_from_django_qs(interviews))

In [4]:
gt = ground_truth_interviews("kellyanne conway")

In [5]:
def get_fps_map(vids):
    vs = Video.objects.filter(id__in=vids)
    return {v.id: v.fps for v in vs}

def frame_second_conversion(c, mode='f2s'):
    fps_map = get_fps_map(set(c.get_grouped_intervals().keys()))
    
    def second_to_frame(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            curr_bounds = intrvl['bounds']
            print(curr_bounds)
            curr_bounds['t1'] = int(curr_bounds['t1']*fps)
            curr_bounds['t2'] = int(curr_bounds['t2']*fps)
            i2['bounds'] = curr_bounds
            return i2
        return map_fn
    
    def frame_to_second(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            curr_bounds = intrvl['bounds']
            curr_bounds['t1'] = int(curr_bounds['t1']/fps)
            curr_bounds['t2'] = int(curr_bounds['t2']/fps)
            i2['bounds'] = curr_bounds
            return i2
        return map_fn
    
    if mode=='f2s':
        fn = frame_to_second
    if mode=='s2f':
        fn = second_to_frame
    output = {}
    for vid, intervals in c.get_grouped_intervals().items():
        output[vid] = intervals.map(fn(fps_map[vid]))
    return IntervalSetMapping(output)

def frame_to_second_collection(c):
    return frame_second_conversion(c, 'f2s')

def second_to_frame_collection(c):
    return frame_second_conversion(c, 's2f')

In [6]:
def get_person_intrvlcol(person_list=None, video_ids=None, 
                         probability=0.9, face_size=None, stride_face=False, labeler='new',
                         exclude_person=False, granularity='frame', payload_type='shot_id'):
    
    def identity_filter(person_list):
        filter_all = None
        for p in person_list:
            if labeler == 'new':
                filter = Q(labeler__name='face-identity-converted:' +  p) | Q(labeler__name='face-identity:' + p)
            else:
                filter = Q(labeler__name='face-identity-old:' +  p)
            if filter_all is None:
                filter_all = filter
            else:
                filter_all = filter_all | filter
        return filter_all
    
    if stride_face:
        labeler = 'new'
    if type(person_list) == str:
        person_list = [person_list.lower()]
    else:
        person_list = [p.lower() for p in person_list]
    
    faceIDs = FaceIdentity.objects \
              .filter(probability__gt=probability) \
              .annotate(face_size=F("face__bbox_y2") - F("face__bbox_y1")) \
              .annotate(video_id=F("face__frame__video_id")) \
    
    if not stride_face:
        faceIDs = faceIDs.exclude(face__shot__isnull=True)
    else:
        faceIDs = faceIDs.filter(face__frame__shot_boundary=False)
        
    if not person_list is None:
        if not exclude_person:
            faceIDs = faceIDs.filter(identity_filter(person_list)) 
        else:
            faceIDs = faceIDs.exclude(identity_filter(person_list)) 
                
    if not face_size is None:
        faceIDs = faceIDs.filter(face_size__gte=face_size)
        
    if not video_ids is None:
        faceIDs = faceIDs.filter(video_id__in=video_ids)
    
    if not stride_face:
        person_intrvllists = ism_from_django_qs(
            faceIDs.annotate(video_id=F("face__shot__video_id"))
                   .annotate(shot_id=F("face__shot_id"))
                   .annotate(min_frame=F("face__shot__min_frame"))
                   .annotate(max_frame=F("face__shot__max_frame"))
                   .annotate(faceID_id=F("identity_id")),\
            bounds_schema={
                'start': 'min_frame',
                'end': 'max_frame',
                'payload': payload_type
            })
        person_intrvlcol = IntervalSetMapping(person_intrvllists).coalesce(('t1', 't2'), Bounds3D.span)
    else:
        if payload_type == 'shot_id':
            payload_type = 'frame_id'
        person_intrvllists_raw = ism_from_django_qs(
            faceIDs.annotate(video_id=F("face__frame__video_id"))
                   .annotate(frame_id=F("face__frame__number"))
                   .annotate(min_frame=F("face__frame__number"))
                   .annotate(max_frame=F("face__frame__number") + 1)
                   .annotate(faceID_id=F("identity_id")),\
            bounds_schema={
                'start': 'min_frame',
                'end': 'max_frame',
                'payload': payload_type
            })
        # dilate and coalesce
        SAMPLE_RATE = 3
        person_intrvllists = {}
        for video_id, intrvllist in person_intrvllists_raw.items():
            video = Video.objects.filter(id=video_id)[0]
            dilation = int(video.fps * SAMPLE_RATE / 2)
            # TODO Check this
            person_intrvllists[video_id] = intrvllist.dilate(dilation).coalesce(('t1', 't2'), Bounds3D.span).dilate(-dilation)
        person_intrvlcol = IntervalSetMapping(person_intrvllists)
        
    if granularity == 'second':
        person_intrvlcol = frame_to_second_collection(person_intrvlcol)
    
    # TODO port this
#     print("Get {} intervals for person {}".format(count_intervals(person_intrvlcol), 
#                                                   person_list[0]+' ...' if len(person_list) > 1 else person_list[0]))
    return person_intrvlcol

In [7]:
def get_commercial_intrvlcol(video_ids=None, granularity='frame'):
    if video_ids is None:
        commercial_qs = Commercial.objects.all()
    else:
        commercial_qs = Commercial.objects.filter(video_id__in=video_ids)
        
    commercial_intrvllists = ism_from_django_qs(commercial_qs)
    commercial = IntervalSetMapping(commercial_intrvllists)
    if granularity == 'second':
        commercial = frame_to_second_collection(commercial)
    return commercial

In [8]:
def load_intervals(video_ids, person_name, host_list,
                    face_size=0.2, stride_face=False, probability=0.7):

    person_intrvlcol = get_person_intrvlcol(person_name, video_ids=video_ids, face_size=face_size, 
                                            stride_face=stride_face, probability=probability, 
                                            granularity='second', payload_type='face_size')
    
    video_ids = list(person_intrvlcol.get_grouped_intervals().keys())
    host_intrvlcol = get_person_intrvlcol(host_list, video_ids=video_ids, face_size=face_size, \
                                          stride_face=stride_face, probability=probability, 
                                          granularity='second', payload_type='shot_id')
    
    commercial = get_commercial_intrvlcol(video_ids, granularity='second')
    
    return person_intrvlcol, host_intrvlcol, commercial

In [9]:
video_ids = [3952, 529, 4611, 8859, 33541, 19959, 32472, 40203, 15916, 16542]
person_name = "kellyanne conway"
host_list = ['mike huckabee', 'natalie allen', 'doug mckelway', 'brooke baldwin', 'martha maccallum', 'jim pinkerton', 'melissa francis', 'maria molina', 'louis burgdorf', 'fareed zakaria', 'shannon bream', 'alisyn camerota', 'kimberley strassel', "soledad o'brien", 'sanjay gupta', 'jeanine pirro', 'rob schmidt', 'heather childers', 'rick reichmuth', 'ed lavandera', 'thomas roberts', 'andrea tantaros', 'brian kilmeade', 'alan colmes', 'shepard smith', 'uma pemmaraju', 'victor blackwell', 'kathleen parker', 'howard kurtz', 'jesse watters', 'martin bashir', 'kevin corke', 'jon scott', 'keith olbermann', 'eric bolling', 'glenn beck', 'john stossel', 'josé díaz-balart', 'richard lui', 'brian stelter', 'anna kooiman', 'elisabeth hasselbeck', 'poppy harlow', 'bill hemmer', 'kelly wright', 'clayton morris', 'hallie jackson', 'steve kornacki', 'rich lowry', 'robert pollock', 'ed schultz', 'fredricka whitfield', 'andrea mitchell', 'larry king', 'brit hume', 'david asman', 'trace gallagher', 'bob beckel', 'mark halperin', 'john heilemann', 'donny deutsch', 'kate snow', 'willie geist', 'rachel maddow', 'toure neblett', 'al sharpton', 'piers morgan', 'leland vittert', 'elliot spitzer', 'joy-ann reid', 'savannah guthrie', 'dylan ratigan', 'ali velshi', 'bret baier', 'chris jansing', 'daniel henninger', 'greg gutfeld', 'chris hayes', 'jillian mele', 'jake tapper', 'trish regan', 'chris matthews', 'nicole wallace', 'carol costello', 'john king', 'alex witt', 'megyn kelly', 'steve doocy', 'stephanie ruhle', 'tucker carlson', 'anthony bourdain', 'ainsley earhardt', 'brian shactman', 'joe scarborough', 'sean hannity', 'james taranto', 'lauren green', 'jj ramberg', 'juan williams', 'greta van susteren', 'cal thomas', "lawrence o'donnell", 'rob schmitt', 'frances rivera', 'neil cavuto', 'christi paul', 'arthel neville', 'touré neblett', 'dana perino', 'michael smerconish', 'krystal ball', 'maria bartiromo', 'john fund', 'john berman', 'brian williams', 'ellis henican', 'abby huntsman', 'ana cabrera', 'erin burnett', 'ari melber', 'pete hegseth', 'eric shawn', 'tom shillue', 'ed henry', "bill o'reilly", 'mika brzezinski', 'chris cuomo', 'peter doocy', 'dagen mcdowell', 'tamron hall', 'jose diaz-balart', 'chris wallace', 'cheryl casone', 'wolf blitzer', 'christiane amanpour', 'laura ingraham', 'alex wagner', 'anderson cooper', 'brenda buttner', 'elizabeth prann', 'sandra smith', 'janice dean', 'campbell brown', 'kate bolduan', 'ayman mohyeldin', 'dave briggs', 'kimberly guilfoyle', 'craig melvin', 'lauren ashburn', 'molly henneberg', 'ronan farrow', 'alicia henley', 'yasmin vossoughian', 'julie banderas', 'bryan llenas', 'melissa harris-perry', 'don lemon', 'dorothy rabinowitz', 'judith miller', 'geraldo rivera', 'jedediah bila', 'chuck todd', 'katy tur', 'christine romans', 'harris faulkner', 'kasie hunt', 'gretchen carlson', 'jane hall', 'jason riley', 'griff jenkins', 'rick folbaum']
person_intrvlcol, host_intrvlcol, commercial = load_intervals(video_ids, person_name, host_list)

NameError: name 'Bounds3D' is not defined

In [10]:
sandbox_videos = [529, 763, 2648, 3459, 3730, 3769, 3952, 4143, 4611, 5281, 6185, 7262, 8220,
    8697, 8859, 9215, 9480, 9499, 9901, 10323, 10335, 11003, 11555, 11579, 11792,
    12837, 13058, 13141, 13247, 13556, 13827, 13927, 13993, 14482, 15916, 16215,
    16542, 16693, 16879, 17458, 17983, 19882, 19959, 20380, 20450, 23181, 23184,
    24193, 24847, 24992, 25463, 26386, 27188, 27410, 29001, 31378, 32472, 32996,
    33004, 33387, 33541, 33800, 34359, 34642, 36755, 37107, 37113, 37170, 38275,
    38420, 40203, 40856, 41480, 41725, 42756, 45472, 45645, 45655, 45698, 48140,
    49225, 49931, 50164, 50561, 51175, 52075, 52749, 52945, 53355, 53684, 54377,
    55711, 57384, 57592, 57708, 57804, 57990, 59122, 59398, 60186]

In [11]:
def load_index(index_dir: str) -> Tuple[CaptionIndex, Documents, Lexicon]:
    print('Loading caption index: please wait...')
    documents = Documents.load(path.join(index_dir, 'docs.list'))
    #TODO fix for the latest version of caption-index
    lexicon = Lexicon.load(path.join(index_dir, 'words.lex'))
#                            lazy_lemmas=False)
    index = CaptionIndex(path.join(index_dir, 'index.bin'),
                         lexicon, documents)
    return index, documents, lexicon

In [12]:
def split_intrvlcol(intrvlcol, seg_length):
    intrvllists_split = {}
    for video_id, intrvllist in intrvlcol.get_grouped_intervals().items():
        intervals_split = []
        for i in intrvllist.get_intervals():
            duration = i['bounds']['t2'] - i['bounds']['t1']
            start = i['bounds']['t1']
            while duration > 0:
                if duration > seg_length:
                    i2 = i.copy()
                    i2['bounds']['t1'] = start
                    i2['bounds']['t2'] = start + seg_length
                    intervals_split.append(i2)
                    duration -= seg_length
                    start += seg_length
                else:
                    i2 = i.copy()
                    i2['bounds']['t1'] = start
                    i2['bounds']['t2'] = start + duration
                    intervals_split.append(i2)
                    duration = 0
        intrvllists_split[video_id] = IntervalSet(intervals_split)
    return IntervalSetMapping(intrvllists_split)

In [13]:
def interview_query(person_intrvlcol, host_intrvlcol, commercial, num_face_intrvlcol=None):
    
    host_intrvlcol_related = {}
    for video_id in person_intrvlcol.get_grouped_intervals():
        if video_id in host_intrvlcol.get_grouped_intervals():
            host_intrvlcol_related[video_id] = host_intrvlcol.get_grouped_intervals()[video_id]
    host_intrvlcol = IntervalSetMapping(host_intrvlcol_related)
    
    # Split long segments into connected pieces
    host_intrvlcol = split_intrvlcol(host_intrvlcol, seg_length=30)
    # person_intrvlcol = split_intrvlcol(person_intrvlcol, seg_length=30)

    # This temporal predicate defines A overlaps with B, or A before by less than 60 seconds,
    #   or A after B by less than 60 seconds
    from rekall.predicates import or_pred, overlaps, before, after
    overlaps_before_or_after_pred = or_pred(
            or_pred(overlaps(), before(max_dist=60)),
            after(max_dist=60))
    
    # debug this
    interview_candidates = host_intrvlcol \
                .join(person_intrvlcol, predicate=overlaps_before_or_after_pred) \
                .coalesce(('t1', 't2'), Bounds3D.span) 
    
    # Sequences may be interrupted by shots where the guest or host don't
    # appear, so dilate and coalesce to merge neighboring segments
    interviews = interview_candidates \
            .dilate(120) \
            .coalesce(('t1', 't2'), Bounds3D.span) \
            .dilate(-1 * 120) \
            .filter_length(min_length=240) \
            .minus(commercial) \
            .filter_length(min_length=240)
    
    
    # remove interview segments which the total person time proportion is below threshold
    def filter_person_time(i):
        # Thresh for Trump 0.4, 0.4, 0.5 
        person_time = 0
        small_person_time, large_person_time = 0, 0
        for height, duration in i.payload:
            person_time += duration
            small_person_time += duration if height < 0.3 else 0
            large_person_time += duration if height > 0.3 else 0
        seg_length = i.end - i.start
        return person_time / seg_length > 0.35 and small_person_time / person_time < 0.7
    
    interviews_person_time = interviews.join(person_intrvlcol,
                             predicate=overlaps(),
                             merge_op=lambda i1, i2: [(i1.start, i1.end, [(i2.payload, i2.end - i2.start)])] ) \
                             .coalesce(payload_plus)

    interviews = interviews_person_time.filter(filter_person_time)
    
    person_all = interviews.overlaps(person_intrvlcol)

    person_with_host = person_all.overlaps(host_intrvlcol)
    
    person_only = person_all.minus(person_with_host)
    
    host_only = interviews.overlaps(host_intrvlcol).minus(person_with_host)
    
    return interviews, person_only, host_only, person_with_host

In [ ]:
def test_interview(person_name):
    if person_name == 'donald trump':
        gt = pickle.load(open('/app/data/interview_gt.pkl', 'rb'))
        gt = VideoIntervalCollection({id: IntervalList(list) for id, list in gt.items()})
        gt = intrvlcol_second2frame(gt)
    else:
        gt = ground_truth_interviews(person_name)
    
    video_ids=list(gt.get_grouped_intervals().keys())
    print(video_ids)
    g
    index, documents, lexicon = load_index("../data/index10a-new-tokenizer")
    print(documents)

    host_list = [h.name for s in CanonicalShow.objects.all() for h in s.hosts.all() ]
    host_list = list(set(host_list))
    print(host_list)
    
    person_intrvlcol, host_intrvlcol, commercial = load_intervals(video_ids, person_name, host_list)
#     return person_intrvlcol, host_intrvlcol, commercial, gt
    

    interviews, person_only, host_only, person_with_host = interview_query(person_intrvlcol, host_intrvlcol, commercial)
#     return interviews, person_in_interviews
    
    for x in interviews.intervals:
        print(x, interviews.intervals[x])
        
    
    print("Total length: %.1fh" % (count_duration(intrvlcol_frame2second(gt)) / 3600))
    print_statistics(intrvlcol_second2frame(interviews).get_allintervals(), gt.get_allintervals())

    summarize_result = intrvllists_to_result(gt.get_allintervals(), color='green')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(commercial).get_allintervals(), color='gray')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(person_intrvlcol).get_allintervals(), color='blue')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(host_intrvlcol).get_allintervals(), color='purple')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(person_with_host).get_allintervals(), color='orange')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(interviews).get_allintervals(), color='red')
    return summarize_result

In [15]:
summarize_result_k = test_interview('kellyanne conway')
# esper_widget(summarize_result_k)

[3952, 529, 4611, 8859, 33541, 19959, 32472, 40203, 15916, 16542]
/app/notebooks
Loading caption index: please wait...
['leland vittert', 'neil cavuto', 'gretchen carlson', 'robert pollock', 'kimberley strassel', 'juan williams', 'thomas roberts', 'sandra smith', 'tucker carlson', 'chris wallace', 'shepard smith', 'don lemon', 'daniel henninger', 'katy tur', 'elliot spitzer', 'willie geist', 'victor blackwell', 'richard lui', 'christiane amanpour', 'nicole wallace', 'ed schultz', 'josé díaz-balart', "lawrence o'donnell", 'mike huckabee', 'maria molina', 'rob schmidt', 'greta van susteren', 'rachel maddow', "soledad o'brien", 'chris cuomo', 'alex wagner', 'shannon bream', 'john fund', 'brit hume', 'alan colmes', 'steve doocy', 'dave briggs', 'glenn beck', 'steve kornacki', 'jon scott', 'ari melber', 'doug mckelway', 'uma pemmaraju', 'fredricka whitfield', 'mark halperin', 'natalie allen', 'martin bashir', 'anderson cooper', 'kate snow', 'piers morgan', 'campbell brown', 'julie banderas'

NameError: name 'Bounds3D' is not defined

In [ ]:
esper_widget(summarize_result_k)